# Effective Tensorflow
Following: https://github.com/vahidk/EffectiveTensorflow/tree/v1

**First example**

In [18]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np

In [4]:
x = tf.random_normal([10, 10])

y = tf.random_normal([10, 10])
z = tf.matmul(x, y)

In [ ]:
sess = tf.Session()
z_val = sess.run(z)

**Parametric curve fitting**

In [116]:
tf.reset_default_graph()

In [40]:
# Placeholders are used to feed values from python to TensorFlow ops. We define
# two placeholders, one for input feature x, and one for output y.
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

# Assuming we know that the desired function is a polynomial of 2nd degree, we
# allocate a vector of size 3 to hold the coefficients. The variable will be
# automatically initialized with random noise.
w = tf.get_variable("w", shape=[3, 1])

# We define yhat to be our estimate of y.
f = tf.stack([tf.square(x), x, tf.ones_like(x)], 1)
yhat = tf.squeeze(tf.matmul(f, w), 1)

In [45]:
# The loss is defined to be the l2 distance between our estimate of y and its
# true value. We also added a shrinkage term, to ensure the resulting weights
# would be small.
loss = tf.nn.l2_loss(yhat - y) + 0.1 * tf.nn.l2_loss(w)

# We use the Adam optimizer with learning rate set to 0.1 to minimize the loss.
train_op = tf.train.AdamOptimizer(0.1).minimize(loss)

def generate_data():
    x_val = np.random.uniform(-10.0, 10.0, size=100)
    y_val = 5 * np.square(x_val) + 3
    return x_val, y_val

sess = tf.Session()
# Since we are using variables we first need to initialize them.
sess.run(tf.global_variables_initializer())
for _ in range(1000):
    x_val, y_val = generate_data()
    _, loss_val = sess.run([train_op, loss], {x: x_val, y: y_val})
    #print(loss_val)
print(sess.run([w]))

[array([[ 4.9894333e+00],
       [-4.0429126e-04],
       [ 3.6196377e+00]], dtype=float32)]


**Setting parameters**

In [87]:
def get_shape(tensor):
    static_shape = tensor.shape.as_list()
    dynamic_shape = tf.unstack(tf.shape(tensor))
    dims = []
    for s in zip(static_shape, dynamic_shape):
        print(s)
        dims.append(s[1] if s[0] is None else s[0])
          
    return dims

In [88]:
get_shape(a)

(None, <tf.Tensor 'unstack_3:0' shape=() dtype=int32>)
(128, <tf.Tensor 'unstack_3:1' shape=() dtype=int32>)


[<tf.Tensor 'unstack_3:0' shape=() dtype=int32>, 128]

**Parameter assignation and variables**

In [125]:
a = tf.Variable(1)
b = tf.constant(2)
assign = tf.assign(a, a + b)

sess = tf.Session()
sess.run(fetches = tf.global_variables_initializer())
print(sess.run(assign))

3


**Control flows**

In [140]:
a = tf.constant([1, 1])
b = tf.constant([2, 2])

p = tf.constant([True, False])

x = tf.where(p, a + b, a * b)

print(tf.Session().run(x))

[3 2]


In [167]:
n = tf.constant(5)

def cond(i, a, b):
    return i < n

def body(i, a, b):
    return i + 1, b, a + b

i, a, b = tf.while_loop(cond, body, (2, 1, 1))

print(tf.Session().run([i,a,b]))

[5, 3, 5]


In [170]:
n = tf.constant(6)

def cond(i, a, b, c):
    return i < n

def body(i, a, b, c):
    return i + 1, b, a + b, tf.concat([c, [a + b]], 0)

i, a, b, c = tf.while_loop(cond, body, (2, 1, 1, tf.constant([1, 1])),
                          shape_invariants=(tf.TensorShape([]),
                          tf.TensorShape([]),
                          tf.TensorShape([]),
                          tf.TensorShape([None])))

print(tf.Session().run(c))

[1 1 2 3 5 8]


In [150]:
tf.reset_default_graph()